no letter at end = after kNearest was run<br>
a is after official neighbourhoods by Id was created <br>
b is after errors list was created, Group 1 to 4 <br>
c is after accuracy was updated <br>

In [60]:
filename = "YELP-Test-650-1-c.pkl"
dill.load_session(filename)

EOFError: Ran out of input

In [59]:
# %load "YELP-Functions.py"
#!/usr/bin/env python

# # YELP Project 2019: Reviews Analysis for Classifying Businesses into Neighbourhoods in Toronto
# ### Overall Question(s): Can language distinguish groups of businesses/people? What in the language? Why?

# In[3]:


import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from tqdm import tqdm
from collections import *
import operator
import itertools
import dill
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
from shapely.geometry import Point, Polygon
import math
from scipy import sparse
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 500)

def everyWord(file, stop_words):
    everyWord_counter = Counter()
    for comment in tqdm(file['1'].values):
        comment = [t.lower() for t in comment.replace('.',' ').replace(',',' ').split(" ") if ((len(t) > 1) and (t.lower() not in stop_words))]
        everyWord_counter.update(comment)
    return everyWord_counter

def topktermFreq(everyWord, stop_words, k):
    AllWords_stop = Counter(everyWord)
    sorted_dict = OrderedDict(sorted(AllWords_stop.items(), key=operator.itemgetter(1), reverse=True))
    topktermFreq = dict(itertools.islice(sorted_dict.items(), k))
    return topktermFreq

def topkdocFreq(file, stop_words, k):
    allWordsinFile = everyWord(file, stop_words)
    topkTF = topktermFreq(allWordsinFile, stop_words, k)
    topkdocFreq = defaultdict(int)
    for comment in tqdm(file['1'].values):
        comment = set(t.lower() for t in comment.replace('.',' ').replace(',',' ').split(" ") if len(t) > 1 and (t.lower() not in stop_words))
        for word in comment:
            if word in topkTF.keys():
                topkdocFreq[word] += 1
            else:
                continue
    return topkdocFreq

def TFIDF_xtrain(xtrain_file, topkDF, stop_words):
    row = []
    col = []
    data = []
    # for each of the comments (rows)
    row_index = 0
    # loop through each of the comments in the 
    for comment in tqdm(xtrain_file['1'].values):
        #comment contains all the words in the comments, but we are only interested in the 15
        comment = [t.lower() for t in comment.replace('.',' ').replace(',',' ').split(" ") if ((len(t) > 1) and (t.lower() not in stop_words))]
        # create a dictionary for all words
        c_counter = Counter(comment)        
        col_index = 0
        # loop through the top 10k words
        for word in topkDF.keys():
            if word in c_counter.keys():
                row.append(row_index)
                col.append(col_index)
                data.append(round(((c_counter[word]/len(comment))*math.log10(len(xtrain_file)/topkDF[word])),5))
                col_index += 1
            else:
                col_index += 1
        row_index += 1
    return sparse.coo_matrix((data,(row,col)), shape = (len(xtrain_file),len(topkDF))).toarray()

def TFIDF_test(xtrain_file, test_file, topkDF, stop_words):
    row = []
    col = []
    data = []
    # for each of the comments (rows)
    row_index = 0
    # loop through each of the comments in the 
    for comment in tqdm(test_file['1'].values):      
        #comment contains all the words in the comments, but we are only interested in the 15
        comment = [t.lower() for t in comment.replace('.',' ').replace(',',' ').split(" ") if ((len(t) > 1) and (t.lower() not in stop_words))]
        # create a dictionary for all words
        c_counter = Counter(comment)  
        col_index = 0
        # loop through the top 10k words
        for word in topkDF.keys():
            if word in c_counter.keys():
                row.append(row_index)
                col.append(col_index)
                data.append(round(((c_counter[word]/len(comment))*math.log10(len(xtrain_file)/topkDF[word])),5))
                col_index += 1
            else:
                col_index += 1
        row_index += 1
    return sparse.coo_matrix((data,(row,col)), shape = (len(test_file),len(topkDF))).toarray()


def existing_neighbourhood_dictionary(businesses_file, neighbourhoods_file):
    id=[]
    latitude=[]
    longitude=[]
    for x in range(businesses_file.shape[0]):
        id.append(businesses_file.iloc[x,13])
        longitude.append(businesses_file.iloc[x,30])
        latitude.append(businesses_file.iloc[x,31])  
    df = pd.DataFrame(
        {'ID': id,
         'Latitude': latitude,
         'Longitude': longitude})
    gdf = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.Longitude, df.Latitude))    
    business_neighbourhood = {}
    b_names = businesses_file.iloc[:,13]
    for key in b_names:
        business_neighbourhood[key] = float('nan')      
    for i in range(gdf.shape[0]): #for businesses
        for j in range(neighbourhoods_file.shape[0]): # for neighbourhood
            if (neighbourhoods_file.loc[j, 'geometry']).contains(gdf.iloc[i,3]) == True:
                business_neighbourhood[businesses_file.iloc[i,13]] = neighbourhoods_file.iloc[j,6]
    return business_neighbourhood

def official_neighbourhoods(businesses_file):
    neighbourhood_official = {}
    for x in tqdm(range(businesses_file.shape[0])):
        neighbourhood_official[(businesses_file.iloc[x,13])] = businesses_file.iloc[x,29]
    return neighbourhood_official

def cosineSimilarity(tfidf_xtrain, tfidf_test):
    similarities = np.zeros((len(tfidf_test),len(tfidf_xtrain)))
    for i in tqdm(range(tfidf_test.shape[0])):
        a = tfidf_test[i]
        for j in range(tfidf_xtrain.shape[0]):
                b = tfidf_xtrain[j]
                cosinesimilarity = round(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)), 1)
                if cosinesimilarity == 0:
                    continue
                else:
                    similarities[i][j] = cosinesimilarity
    return similarities

def kNearestNeighbours(test_file, xtrain_comments_file, existing_neighbourhoods_xtrain, cos_sim, k): #want to specify the size  
    kNearest = {}   
    # Have a dictionary stating the business and the neighbourhood, for ex. {2: 'A', 3: 'C', 6: 'C'}
    for row in tqdm(range(len(cos_sim))):
        possible_neighbours = []
        for col in range(len(cos_sim[0])):
            possible_neighbours.append((cos_sim[row][col], existing_neighbourhoods_xtrain[xtrain_comments_file.iloc[col,1]]))
        possible_neighbours.sort(key=lambda x: x[0], reverse=True)
  
        votes = Counter()
        for index in range(k):
            votes[possible_neighbours[index][1]] += 1
    
        kNearest[test_file.iloc[row,1]] = votes.most_common(1)[0][0]
    return kNearest
        
def Accuracy(kNearest_, existing_neighbourhood_dictionary):
    total = len(kNearest_)
    sum = 0
    for key in kNearest_.keys():
        if kNearest_[key] == existing_neighbourhood_dictionary[key]:
            sum += 1
    average = round(sum/total,5)
    return average



In [ ]:
# %load YELP-files.py
#!/usr/bin/env python

stop_words =  list(stopwords.words('english')) 
businesses = pd.read_csv("businesses (1).csv")
data = pd.read_csv('business_text_stripped.csv')
toronto_map = gpd.read_file('Neighbourhoods.geojson')

# TEST CONDITIONS: <br>
### test size = 0.33 (Xtrain = 21,454 ; Xtest = 10568)
### 650 WORDS

In [ ]:
X_train, X_test = train_test_split(data, test_size=0.33)

In [ ]:
topkDF_650 = topkdocFreq(X_train, stop_words, 650)

In [ ]:
tfidf_xtrain_650 = TFIDF_xtrain(X_train, topkDF_650, stop_words)

In [ ]:
tfidf_test_650 = TFIDF_test(X_train, X_test, topkDF_650, stop_words)

In [ ]:
cos_sim_650 = cosineSimilarity(tfidf_xtrain_650, tfidf_test_650)

In [4]:
official_neighbourhoods_byName = official_neighbourhoods(businesses)

100%|██████████| 32123/32123 [00:01<00:00, 18971.84it/s]


### Splitting into 4 Test Groups

In [ ]:
testGroup_1, testGroup_2, testGroup_3, testGroup_4 = np.array_split(X_test, 4)

In [ ]:
cos_sim_1, cos_sim_2, cos_sim_3, cos_sim_4 = np.array_split(cos_sim_650, 4)

# Nearest Neighbours

### >>> Group 1

How many are supposed to be Nan? 557 out of 2642 don't even belong to any of the neighbourhoods listed. How do we identify these?

In [21]:
kNearest_650_1 = kNearestNeighbours(testGroup_1, X_train, official_neighbourhoods_byName, cos_sim_1, 50)

100%|██████████| 2642/2642 [39:37<00:00,  1.61it/s]  


In [15]:
accuracy_650_1 = Accuracy(kNearest_650_1, official_neighbourhoods_byName)

In [16]:
accuracy_650_1

0.0564

In [43]:
errors_1 = []
for key in tqdm(kNearest_650_1.keys()):
    if kNearest_650_1[key] != official_neighbourhoods_byName[key] and isinstance(official_neighbourhoods_byName[key],str) == True:
        errors_1.append(("Prediction: " + str(kNearest_650_1[key]) +  " but actually: " + official_neighbourhoods_byName[key]))

100%|██████████| 2642/2642 [00:00<00:00, 240105.55it/s]


### >>> Group 2

In [5]:
kNearest_650_2 = kNearestNeighbours(testGroup_2, X_train, official_neighbourhoods_byName, cos_sim_2, 50)

100%|██████████| 2642/2642 [24:23<00:00,  1.77it/s]


In [17]:
accuracy_650_2 = Accuracy(kNearest_650_2, official_neighbourhoods_byName)

In [18]:
accuracy_650_2

0.05148

In [44]:
errors_2 = []
for key in tqdm(kNearest_650_2.keys()):
    if kNearest_650_2[key] != official_neighbourhoods_byName[key] and isinstance(official_neighbourhoods_byName[key],str) == True:
        errors_2.append(("Prediction: " + str(kNearest_650_2[key]) +  " but actually: " + official_neighbourhoods_byName[key]))

100%|██████████| 2642/2642 [00:00<00:00, 203197.05it/s]


### >>> Group 3

In [36]:
kNearest_650_3 = kNearestNeighbours(testGroup_3, X_train, official_neighbourhoods_byName, cos_sim_3, 50)

100%|██████████| 2642/2642 [27:52<00:00,  1.68it/s]


In [19]:
accuracy_650_3 = Accuracy(kNearest_650_3, official_neighbourhoods_byName)

In [20]:
accuracy_650_3

0.04921

In [45]:
errors_3 = []
for key in tqdm(kNearest_650_3.keys()):
    if kNearest_650_3[key] != official_neighbourhoods_byName[key] and isinstance(official_neighbourhoods_byName[key],str) == True:
        errors_3.append(("Prediction: " + str(kNearest_650_3[key]) +  " but actually: " + official_neighbourhoods_byName[key]))

100%|██████████| 2642/2642 [00:00<00:00, 240121.15it/s]


### >>> Group 4

In [49]:
kNearest_650_4 = kNearestNeighbours(testGroup_4, X_train, official_neighbourhoods_byName, cos_sim_4, 50)

100%|██████████| 2642/2642 [27:45<00:00,  1.61it/s]


In [21]:
accuracy_650_4 = Accuracy(kNearest_650_4, official_neighbourhoods_byName)

In [22]:
accuracy_650_4

0.04693

In [53]:
errors_4 = []
for key in tqdm(kNearest_650_4.keys()):
    if kNearest_650_4[key] != official_neighbourhoods_byName[key] and isinstance(official_neighbourhoods_byName[key],str) == True:
        errors_4.append(("Prediction: " + str(kNearest_650_4[key]) +  " but actually: " + official_neighbourhoods_byName[key]))

100%|██████████| 2642/2642 [00:00<00:00, 220099.53it/s]


Graphs of: <br>
how many were Nan vs how many were supposed to be nan <br>
comparing what the distribution should have been (bar chart) /each bar is a neighbourhood <br>
comparing what the distrubution actually was (bar chart) <br>
list ones that were correctly identified 

In [29]:
total_nan = 0
total_nan_test = 0
for key in kNearest_650_1.keys():
    if isinstance(official_neighbourhoods_byName[key],str) == False:
        total_nan += 1
    if isinstance(kNearest_650_1[key],str) == False:
        total_nan_test += 1

for key in kNearest_650_2.keys():
    if isinstance(official_neighbourhoods_byName[key],str) == False:
        total_nan += 1
    if isinstance(kNearest_650_2[key],str) == False:
        total_nan_test += 1

for key in kNearest_650_3.keys():
    if isinstance(official_neighbourhoods_byName[key],str) == False:
        total_nan += 1
    if isinstance(kNearest_650_3[key],str) == False:
        total_nan_test += 1

for key in kNearest_650_4.keys():
    if isinstance(official_neighbourhoods_byName[key],str) == False:
        total_nan += 1
    if isinstance(kNearest_650_4[key],str) == False:
        total_nan_test += 1
print(total_nan)
print(total_nan_test)

2380
1


Comparison Bar Chart information

For actual answers, distribution:

In [61]:
answerkey_neighbourhoods = []
for key in tqdm(kNearest_650_1.keys()):
    if isinstance(official_neighbourhoods_byName[key],str) == True:
        answerkey_neighbourhoods.append(official_neighbourhoods_byName[key])
for key in tqdm(kNearest_650_2.keys()):
    if isinstance(official_neighbourhoods_byName[key],str) == True:
        answerkey_neighbourhoods.append(official_neighbourhoods_byName[key])
for key in tqdm(kNearest_650_3.keys()):
    if isinstance(official_neighbourhoods_byName[key],str) == True:
        answerkey_neighbourhoods.append(official_neighbourhoods_byName[key])
for key in tqdm(kNearest_650_4.keys()):
    if isinstance(official_neighbourhoods_byName[key],str) == True:
        answerkey_neighbourhoods.append(official_neighbourhoods_byName[key])

100%|██████████| 2642/2642 [00:00<00:00, 330155.86it/s]


In [62]:
counting_answerkey = Counter(answerkey_neighbourhoods)

In [45]:
len(counting_answerkey)

140

For the correct neigbourhoods, can see what the distribution is.

In [31]:
correct_neighbourhoods = []
for key in tqdm(kNearest_650_1.keys()):
    if kNearest_650_1[key] == official_neighbourhoods_byName[key]:
        correct_neighbourhoods.append(official_neighbourhoods_byName[key])
for key in tqdm(kNearest_650_2.keys()):
    if kNearest_650_2[key] == official_neighbourhoods_byName[key]:
        correct_neighbourhoods.append(official_neighbourhoods_byName[key])
for key in tqdm(kNearest_650_3.keys()):
    if kNearest_650_3[key] == official_neighbourhoods_byName[key]:
        correct_neighbourhoods.append(official_neighbourhoods_byName[key])
for key in tqdm(kNearest_650_4.keys()):
    if kNearest_650_4[key] == official_neighbourhoods_byName[key]:
        correct_neighbourhoods.append(official_neighbourhoods_byName[key])

100%|██████████| 2642/2642 [00:00<00:00, 56213.99it/s]


In [32]:
counting_correct = Counter(correct_neighbourhoods)

In [47]:
len(counting_correct)

27

For test case neighbourhood answers: 

In [48]:
test_neighbourhoods = []
for key in tqdm(kNearest_650_1.keys()):
    test_neighbourhoods.append(kNearest_650_1[key])
for key in tqdm(kNearest_650_2.keys()):
    test_neighbourhoods.append(kNearest_650_2[key])
for key in tqdm(kNearest_650_3.keys()):
    test_neighbourhoods.append(kNearest_650_3[key])
for key in tqdm(kNearest_650_4.keys()):
    test_neighbourhoods.append(kNearest_650_4[key])

100%|██████████| 2642/2642 [00:00<00:00, 659839.89it/s]


In [49]:
counting_test = Counter(test_neighbourhoods)

In [52]:
len(counting_test)

90